# MedGemma 1.5 W4A8 量化 + RadGraph F1 评估

## 量化说明
- **W4A8**：4-bit 权重 + 8-bit 激活
- **预期**：F1 接近原始，显存适中降低
- **前置**：需先运行 01_original 获取基线

## 环境说明
- **Kaggle P100**：PyTorch cu118
- **transformers 4.47.1 + bitsandbytes**
- **运行流程**：安装 → Restart Session → Run All

# Cell 1: 安装（必须先运行，完成后 Restart Session）

In [ ]:
# W4A8 量化环境
import subprocess, sys, os, shutil

TF_ENV = "/kaggle/working/tf_env"
CONSTRAINTS = "/kaggle/working/constraints.txt"

if os.path.exists(TF_ENV):
    shutil.rmtree(TF_ENV)

print("使用 Kaggle 预装的 PyTorch...")

subprocess.run(["pip", "install", "-q", "bitsandbytes"], check=True)

with open(CONSTRAINTS, "w") as f:
    f.write("transformers==4.47.1\n")

subprocess.run([
    "pip", "install", "--target", TF_ENV, "-q", "-c", CONSTRAINTS,
    "transformers==4.47.1", "pillow<12", "jinja2",
    "-i", "https://pypi.tuna.tsinghua.edu.cn/simple"
], check=True)

subprocess.run([
    "pip", "install", "-q", "radgraph",
    "-i", "https://pypi.tuna.tsinghua.edu.cn/simple"
], check=True)

sys.path.insert(0, TF_ENV)

print("✅ 安装完成，请 Restart Session → Run All")

# Cell 2: HuggingFace 登录

In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

try:
    tok = UserSecretsClient().get_secret("zhuxirui11")
    if tok:
        login(token=tok)
        print("✅ HF 登录成功")
    else:
        print("⚠️ HUGGINGFACE_TOKEN 为空")
except Exception as e:
    print("❌ 未配置 zhuxirui11 Secret")
    print("\n请按以下步骤配置：")
    print("1. 点击右侧 'Add-ons' → 'Secrets'")
    print("2. Label 填写：zhuxirui11")
    print("3. Value 填写你的 HF token")
    print("4. 保存后重新运行本 Cell")

# Cell 3: 环境检查

In [ ]:
import sys, torch

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    # W4A8 激活用 FP16
    DTYPE = torch.float16
    print("Precision: FP16 (W4A8 量化)")
else:
    DTYPE = torch.float32

# Cell 4: 导入与路径配置

In [ ]:
import sys, os, gc
import pandas as pd
from PIL import Image
from tqdm import tqdm

TF_ENV = "/kaggle/working/tf_env"
if TF_ENV not in sys.path:
    sys.path.insert(0, TF_ENV)
elif sys.path[0] != TF_ENV:
    sys.path.remove(TF_ENV)
    sys.path.insert(0, TF_ENV)

DATASET_ROOT = "/kaggle/input/datasets/simhadrisadaram/mimic-cxr-dataset/official_data_iccv_final"
CSV_CANDIDATES = [
    "/kaggle/input/datasets/xiruizhu1111/clean-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/datasets/xiruizhu1111/clean-train-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/clean-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/working/mimic_eval_single_image_final_233.csv",
]

CSV_PATH = None
for p in CSV_CANDIDATES:
    if os.path.exists(p):
        CSV_PATH = p
        break

if not CSV_PATH:
    print("❌ 未找到 CSV 文件！尝试的路径：")
    for p in CSV_CANDIDATES:
        print(f"  - {p}")
    raise FileNotFoundError("CSV 文件不存在")

# 验证 transformers 版本
import transformers
tf_ver = transformers.__version__
if tf_ver.startswith('5.'):
    raise RuntimeError(
        f"❌ transformers {tf_ver} 不兼容！需要 4.47.1\n"
        f"解决方法：重新运行 Cell 1，确保看到 'transformers-4.47.1'，然后 Restart Session"
    )
print(f"✅ transformers: {tf_ver}")

print(f"Dataset: {DATASET_ROOT}")
print(f"✅ CSV: {CSV_PATH}")

# Cell 4.5: 检查 CSV 结构

In [ ]:
import pandas as pd
df_check = pd.read_csv(CSV_PATH)
print(f"✅ CSV: {len(df_check)} 行")
print(f"列名: {list(df_check.columns)}")
print(df_check.head(2))

# Cell 5: 加载 MedGemma W4A8 量化模型

In [ ]:
for k in list(sys.modules.keys()):
    if k == "transformers" or k.startswith("transformers."):
        del sys.modules[k]

from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

model_id = "google/medgemma-1.5-4b-it"
print(f"Loading MedGemma W4A8 (4-bit weight + 8-bit activation)...")

# W4A8 量化配置
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,  # W4A8 用双重量化提升精度
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

if torch.cuda.is_available():
    mem_gb = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"✅ W4A8 model loaded, GPU memory: {mem_gb:.2f} GB")
    torch.cuda.reset_peak_memory_stats()

# Cell 6: 生成报告

In [ ]:
import re

PROMPT_TEMPLATE = "Generate a radiology report for this {view} chest X-ray."

def fix_image_path(path):
    if pd.isna(path) or not path:
        return None
    path = str(path).strip()
    if path.startswith("/kaggle/input/datasets/"):
        return path if os.path.exists(path) else None
    if path.startswith("/kaggle/input/mimic-cxr-dataset/"):
        new_path = path.replace("/kaggle/input/mimic-cxr-dataset/", "/kaggle/input/datasets/simhadrisadaram/mimic-cxr-dataset/")
        return new_path if os.path.exists(new_path) else None
    if not path.startswith("/"):
        full = os.path.join(DATASET_ROOT, path)
        return full if os.path.exists(full) else None
    return path if os.path.exists(path) else None

def get_single_image_path(cell_val):
    if pd.isna(cell_val):
        return None
    s = str(cell_val).strip().replace("[", "").replace("]", "").replace("'", "").replace('"', "").split(",")[0].strip()
    if "files" in s:
        rel = "files" + s.split("files", 1)[1]
    else:
        rel = s.strip("/")
    full = os.path.join(DATASET_ROOT, rel) if not rel.startswith("/") else rel
    return fix_image_path(full)

def generate_report(model, processor, img_path, view="PA"):
    if not os.path.exists(img_path):
        return ""
    try:
        img = Image.open(img_path).convert("RGB")
    except:
        return ""
    prompt = PROMPT_TEMPLATE.format(view=view)
    msgs = [{"role": "user", "content": [{"type": "image", "image": img}, {"type": "text", "text": prompt}]}]
    inp = processor.apply_chat_template(msgs, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt").to(model.device, dtype=DTYPE)
    L = inp["input_ids"].shape[-1]
    with torch.inference_mode():
        out = model.generate(
            **inp,
            max_length=None,  # 禁用默认 max_length
            max_new_tokens=300,  # 使用相对长度
            min_new_tokens=5,  # 强制最少生成
            pad_token_id=0,  # 显式设置 pad token
            do_sample=False
        )
    txt = processor.decode(out[0][L:], skip_special_tokens=True)
    return re.sub(r'\s+', ' ', txt.replace("Findings:", "").replace("Impression:", "")).strip()

df = pd.read_csv(CSV_PATH)
GT_COL = "Ground_Truth" if "Ground_Truth" in df.columns else "text"
IMG_COL = "Image_Path" if "Image_Path" in df.columns else None

rows_out = []
NUM = min(50, len(df))

for idx, row in tqdm(df.head(NUM).iterrows(), total=NUM, desc="Generating W4A8 reports"):
    path, view = None, "PA"
    if IMG_COL:
        path = fix_image_path(row.get(IMG_COL))
        view = row.get("View", "PA")
    else:
        for c, v in [("PA", "PA"), ("AP", "AP"), ("Lateral", "Lateral")]:
            if c in df.columns and (p := get_single_image_path(row.get(c))):
                path, view = p, v
                break
    if not path:
        continue
    gt = str(row.get(GT_COL) or "").strip()
    if not gt or gt.startswith("You are"):
        continue
    rep = generate_report(model, processor, path, view)
    rows_out.append({
        "subject_id": row["subject_id"],
        "View": view,
        "Image_Path": path,
        "Ground_Truth": gt,
        "Generated_Report": rep
    })

df_sub = pd.DataFrame(rows_out)
print(f"\n✅ Generated {len(df_sub)} reports")

# Cell 7a: RadGraph 兼容性修复（必须先运行！）

**说明**：修复 `transformers` 5.x 与 `radgraph` 的兼容性问题

In [ ]:
from transformers import BertTokenizer

fixed_methods = []

# 1. 添加 encode_plus 方法
if not hasattr(BertTokenizer, 'encode_plus'):
    def encode_plus_wrapper(self, text, *args, **kwargs):
        return self(text, *args, **kwargs)
    BertTokenizer.encode_plus = encode_plus_wrapper
    fixed_methods.append('encode_plus')

# 2. 添加 build_inputs_with_special_tokens 方法
if not hasattr(BertTokenizer, 'build_inputs_with_special_tokens'):
    def build_inputs_with_special_tokens_wrapper(self, token_ids_0, token_ids_1=None):
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        return [self.cls_token_id] + token_ids_0 + [self.sep_token_id] + token_ids_1 + [self.sep_token_id]
    BertTokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens_wrapper
    fixed_methods.append('build_inputs_with_special_tokens')

# 3. 添加 get_special_tokens_mask 方法
if not hasattr(BertTokenizer, 'get_special_tokens_mask'):
    def get_special_tokens_mask_wrapper(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        if already_has_special_tokens:
            return super(BertTokenizer, self).get_special_tokens_mask(
                token_ids_0=token_ids_0, token_ids_1=token_ids_1, already_has_special_tokens=True
            )
        if token_ids_1 is None:
            return [1] + ([0] * len(token_ids_0)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
    BertTokenizer.get_special_tokens_mask = get_special_tokens_mask_wrapper
    fixed_methods.append('get_special_tokens_mask')

if fixed_methods:
    print(f"✅ RadGraph 兼容性修复已应用: {', '.join(fixed_methods)}")
else:
    print("✅ BertTokenizer 已有所有必需方法")

# Cell 7b: RadGraph F1 评估

**指标说明**（百分制）：
- **RG_E**: Entity F1（实体匹配）
- **RG_ER**: Entity + Relation F1（实体+关系，**论文常用指标**）
- **RG_ER_bar**: Complete Match F1（完全匹配）

In [ ]:
from radgraph import F1RadGraph

print("🔍 开始 RadGraph F1 评估...")
print(f"评估样本数: {len(df_sub)}\n")

# 准备数据
refs = df_sub["Ground_Truth"].fillna("").tolist()
hyps = df_sub["Generated_Report"].fillna("").tolist()

# 过滤掉空报告
valid_pairs = [(h, r) for h, r in zip(hyps, refs) if h and r and len(h.strip()) > 0]
if len(valid_pairs) < len(hyps):
    print(f"⚠️ 跳过 {len(hyps) - len(valid_pairs)} 条空报告\n")
hyps_clean, refs_clean = zip(*valid_pairs) if valid_pairs else ([], [])

# 初始化 RadGraph
f1radgraph = F1RadGraph(reward_level="all")

# 计算 F1
results = f1radgraph(hyps=list(hyps_clean), refs=list(refs_clean))

# 解析返回值：results[0] = (simple, partial, complete)
avg_scores = results[0]
simple_f1 = float(avg_scores[0])    # RG_ER_bar
partial_f1 = float(avg_scores[1])   # RG_ER (论文常用)
complete_f1 = float(avg_scores[2])  # RG_E

# 显示结果（百分制）
print("=" * 60)
print("MedGemma W4A8 RadGraph F1 评估结果（百分制）")
print("-" * 60)
print(f"RG_E (Entity):              {complete_f1*100:.2f}")
print(f"RG_ER (Entity+Relation):    {partial_f1*100:.2f}  ← 论文常用")
print(f"RG_ER_bar (Complete Match): {simple_f1*100:.2f}")
print("=" * 60)

# 保存分数（小数格式）
W4A8_SCORES = {"rg_e": complete_f1, "rg_er": partial_f1, "rg_er_bar": simple_f1}
W4A8_GPU_GB = torch.cuda.max_memory_allocated(0) / (1024**3) if torch.cuda.is_available() else 0
print(f"\nPeak GPU 显存: {W4A8_GPU_GB:.2f} GB")
print("\n✅ 评估完成！")

# Cell 8: 保存结果

In [ ]:
import json

del model, processor
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

df_sub.to_csv("/kaggle/working/w4a8_medgemma_results.csv", index=False)
with open("/kaggle/working/w4a8_scores.json", "w") as f:
    json.dump({"scores": W4A8_SCORES, "gpu_gb": W4A8_GPU_GB}, f)

print("✅ W4A8 结果已保存")
print("\n下一步：运行 04_compare 对比结果")